In [1]:
from langchain_openai import ChatOpenAI 
import os
from load_dotenv import load_dotenv

#This function will load all the variables from the .env file and will
#make them available in the os.environ dictionary (env variables)

load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("Bro API KEY Variable exists")
else :
    raise ValueError("OPENAI API KEY not found")

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

Bro API KEY Variable exists


## **CHAIN WITH Parallel Chains**

In [2]:
#TASK -1 [Prompt]

promt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie summarizer"),
    ("human","please summarize the movie in brief: {input}")
])

In [3]:
#TASK - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

In [4]:
# TASK -3 [Str Parser]

str_parser = StrOutputParser()

In [6]:
# TASK - 4 [Custom Runnable]

from langchain_core.runnables import RunnableLambda

def dictionary_maker(text:str)-> dict:

    return {"text": text}

dictionary_maker_runnable = RunnableLambda(dictionary_maker) 

# *Parallel chain 1*

In [9]:
# TASK - 1 [Prompt]

from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai.chat_models.base import ChatOpenAI
linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Linkedln post generator"),
    ("human", "Create a post for the following text for linkedin: {text}")])

# Task - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

# TASK - 3 [Str Parser]

str_parser = StrOutputParser()

chain_linkedin = linkedin_prompt | llm_openai | str_parser


# *Parallel chain 2*

In [12]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

In [11]:
def insta_chain(text:dict):
    
    text = text["text"]

    
    # TASK - 1 [Prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a instagram post generator"),
    ("human", "Create a post for the following text for instagram: {text}")])

    # Task - 2 [LLM]
    llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

    # TASK - 3 [Str Parser]
    str_parser = StrOutputParser()

    chain_insta = insta_prompt | llm_openai | str_parser

    result = chain_insta.invoke(text)

    return result


insta_chain_runnable = RunnableLambda(insta_chain)









## **Final Orchestration**

In [13]:
final_chain = promt_template | llm_openai | str_parser | dictionary_maker_runnable | RunnableParallel(branches = {"linkedin": chain_linkedin, "instagram": insta_chain_runnable})

In [14]:
final_chain.invoke("transformers")

{'branches': {'linkedin': 'Transformers (2007) — more than a summer blockbuster: it’s a compact case study in leadership, technology and collaboration.\n\nQuick refresher: a teenager, Sam Witwicky, unwittingly buys Bumblebee — an Autobot in disguise — and gets pulled into a global race between Autobots and Decepticons for the AllSpark, a powerful dual‑use technology that can create or destroy. Big battles, big stakes, and big choices.\n\nThree leadership & innovation lessons I took away:\n- Spot potential where others see ordinary: Bumblebee arrives disguised as a used car — a reminder that breakthrough value is often hidden in plain sight.  \n- Lead with purpose and sacrifice: Optimus Prime’s choices show how clarity of mission and willingness to take risks unite teams in crisis.  \n- Treat technology as a responsibility: the AllSpark is a powerful reminder that innovation can be transformative or catastrophic depending on who controls it — governance and ethics matter.\n\nAlso: cross

## **Chain as a Runnable**

In [16]:
# Task - 1 [Beautify Function]

def beautify(final_response:dict)-> dict:
    linkedin_response = final_response['branches']["linkedin"]
    instagram_response = final_response['branches']["instagram"]

    return {"linedin": linkedin_response, "instagram": instagram_response}

beautify_runnable = RunnableLambda(beautify)

# TASK - 2 [Final Chain]

# final_chain

# Beautfied Chain
beautified_chain = final_chain | beautify_runnable

beautified_chain.invoke("Pushpa")

{'linedin': 'Watched Pushpa: The Rise (2021) — a raw, high‑velocity story about ambition, power and survival set in the Seshachalam forests of Andhra Pradesh.\n\nQuick snapshot:\n- Pushpa Raj (Allu Arjun) starts as a lowly porter and climbs to become a feared kingpin in the illegal red‑sandalwood trade.\n- The film is driven by grit, cunning and violence, and balances a personal love story with escalating rivalries and a determined police pursuit.\n- Allu Arjun’s performance and the film’s relentless momentum set the stage for further conflict in a planned sequel.\n\nProfessional takeaways:\n- Rise from the margins: Persistence and street‑smarts can create opportunity, but they must be paired with integrity to sustain long‑term success.\n- Networks scale power: Building a reliable (and ethical) network amplifies reach — systems matter more than solo heroics.\n- The danger of unchecked ambition: Short‑term gains achieved through ruthless tactics often sow the seeds of conflict later.\n-